In [ ]:
#function to trnsapose normal (non numpy) array/matrix
def transpose_mat(matrix):
    row_len = len(matrix[0])
    trans_mat = [[row[i] for row in matrix] for i in range(row_len)]
    return trans_mat

In [ ]:
from matplotlib.image import imread
import matplotlib.pyplot as plt
import numpy as np
import glob
plt.rcParams['figure.figsize'] = [4,4] #define size of picture to be displayed

images = []

for image in glob.glob('imageset1/*.png'): #for all .png images in imageset1 folder
    A = imread(image) #import image
    X = np.mean(A,-1) #Convert to greyscale
    images.append(X) #append grey scale images into array
    img = plt.imshow(X) #show the image
    img.set_cmap('gray')
    plt.axis('off')
    #plt.show()
    
#print(len(images),len(images[0])) #prints dimensions of images matrix
print(images[0].shape)

In [ ]:
#avrgcard = np.mean(images,axis=1)
#X = images - np.tile(avrgcard,(images.shape[1],1)).T  

U, S, VT = np.linalg.svd(X,full_matrices=False) #preform economy SVD on image
S = np.diag(S) #Define sigma as the diagnals

j = 0
for r in (5,20,100):
    #construct apporximate image with given eigens(r)
    Xapprox = U[:,:r] @ S[0:r,:r] @ VT[:r,:] #calculate approx by using only a pecified number of defining elements
    plt.figure(j+1)
    j+=1
    img = plt.imshow(Xapprox) #show the image
    img.set_cmap('gray')
    plt.axis('off')
    plt.title('r = ' + str(r))
    plt.show()

In [ ]:
#calculation of important singular values 
plt.figure(1)
plt.semilogy(np.diag(S))
plt.title('Singular Values') # Diagonal elements of Sigma, the sigma_j values
plt.show()

plt.figure(2)
plt.plot(np.cumsum(np.diag(S))/np.sum(np.diag(S)))
plt.title('Singular Values: Cumulative Sum')
plt.show()

In [7]:
# To combat overfitting, we will perform image data augmentation to create more training data
import Augmentor, os
#-----------Note-----------#
# Augmentor requires blank #
# directory of only images # 
#  no other dirs or files  #
# make another dir called  #
#  "original_images" with  #
# just the png's from set  #
#--------------------------#
datasetPath = os.path.join(os.getcwd(), "imageset1", "original_images")
pipeline = Augmentor.Pipeline(datasetPath) # First create a pipeline object
# Now we can add "operations" to the pipeline -> what effects we apply to each image to augment it
# Operation 1: Rotate somewhere between -10º and 10º determined by a 70% probability
pipeline.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
# Operation 2: Zoom either in by up to 1.5x or out by 0.5x with a probability of 30%
pipeline.zoom(probability=0.3, min_factor=0.5, max_factor=1.5)
pipeline.sample(25) # Create 25 sample images

Initialised with 64 image(s) found.
Output directory set to /Users/liam/Desktop/School/Spring_2023/ECE_172/Playing-Card-Machine-Learning/Playing-Card-Identification-Using-Machine-Learning/imageset1/original_images/output.

Processing <PIL.Image.Image image mode=RGBA size=691x1056 at 0x1107FB850>: 100%|██████████████████| 25/25 [00:03<00:00,  7.07 Samples/s]


In [30]:
# Move photos to their folders
import os, shutil, uuid, glob

def createDatasetFolders():
    # Create folder names
    parentDirectory = os.getcwd()
    suits = ["C", "H", "S", "D"]
    for suit in suits:    
        for i in range(1, 14):
            if i == 1:
                cardNum = "A" # Aces
            elif i == 11:
                cardNum = "J" # Jacks
            elif i == 12:
                cardNum = "Q" # Queens
            elif i == 13:
                cardNum = "K" # Kings
            else:
                cardNum = str(i) # Card num
            path = os.path.join(parentDirectory, "imageset1", cardNum + suit) 
            if not os.path.exists(path):
                # If the folder does not exist, make it
                os.mkdir(path)

def moveImageToFolder(filename):
    # Augmentor library names files like: "imageset1_original_{cardname).png_{gibberish}.png"
    firstPNGIndex = filename.find(".png")
    
    # Use recursion to find the 3rd understore
    def find_nth(string, substring, n):
       if (n == 1):
           return string.find(substring)
       else:
           return string.find(substring, find_nth(string, substring, n - 1) + 1)
    
    thirdUnderscoreIndex = find_nth(filename, "_", 4) # make it 4 if in "original_images" folder
    if firstPNGIndex != -1:
        cardName = filename[thirdUnderscoreIndex+1:firstPNGIndex]
        newDir = os.path.join(os.getcwd(), "imageset1", cardName)
        #print(newDir)
        newLocation = os.path.join(newDir, str(uuid.uuid4()) + ".png")
        # Use shutil to move the file and uuid to generate a unique filename
        #oldLocation = os.path.join(os.getcwd(), "imageset1", "original_images", "output", filename)
        shutil.move(filename, newLocation)
    
#moveImageToFolder("original_images_original_2C.png_a755ce2d-0e0d-42b7-a146-7bd89c6d8744.png")

for file in glob.glob('imageset1/original_images/output/*.png'):
    try:
        #print(file)
        moveImageToFolder(file)
    except Exception as e:
        print(f"\n{e}\n")
        continue


[Errno 2] No such file or directory: '/Users/liam/Desktop/School/Spring_2023/ECE_172/Playing-Card-Machine-Learning/Playing-Card-Identification-Using-Machine-Learning/imageset1/honor_heart-14/acac88c6-96d6-4601-aab4-c95296f73455.png'


[Errno 2] No such file or directory: '/Users/liam/Desktop/School/Spring_2023/ECE_172/Playing-Card-Machine-Learning/Playing-Card-Identification-Using-Machine-Learning/imageset1/yellow_back/2ea92e6d-34ca-4bb3-8b3d-4db0989f5d0f.png'


[Errno 2] No such file or directory: '/Users/liam/Desktop/School/Spring_2023/ECE_172/Playing-Card-Machine-Learning/Playing-Card-Identification-Using-Machine-Learning/imageset1/honor_clubs/e2795aa9-0038-413d-9b37-ec92c4bc6a8e.png'


[Errno 2] No such file or directory: '/Users/liam/Desktop/School/Spring_2023/ECE_172/Playing-Card-Machine-Learning/Playing-Card-Identification-Using-Machine-Learning/imageset1/gray_back/148ccc97-9441-4559-82a7-9ae5a1cfe49c.png'


[Errno 2] No such file or directory: '/Users/liam/Desktop/School/Sprin